In [1]:
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [243]:
def load_languages(path: str, size: int = 12, seperator: str = 'language') -> pd.DataFrame:

    ger = 'German'
    eng = 'English'
    tr = 'Turkish'
    spa = 'Spanish'
    jp = 'Japanese'
    rus = 'Russian'
    fr = 'French'
    ita = 'Italian'
    chi = 'Chinese'
    mak = 'Macedonian'

    df_ = pd.read_csv(path)
    df_['language'] = df_['language'].apply(lambda x: str(x).strip())
    
    df_['language'] = df_['language'].replace('İngilizce', eng)
    df_['language'] = df_['language'].replace('Türkçe', tr)
    df_['language'] = df_['language'].replace('Almanca', ger)
    df_['language'] = df_['language'].replace('ingilizce', eng)
    df_['language'] = df_['language'].replace('İspanyolca', spa)
    df_['language'] = df_['language'].replace('Arapça', 'Arabic')
    df_['language'] = df_['language'].replace('Rusça', rus)
    df_['language'] = df_['language'].replace('Fransızca', fr)
    df_['language'] = df_['language'].replace('İtalyanca', ita)
    df_['language'] = df_['language'].replace('Ingilizce', eng)
    df_['language'] = df_['language'].replace('Kürtçe', 'Kurdish')
    df_['language'] = df_['language'].replace('Korece', 'Korean')
    df_['language'] = df_['language'].replace('Bulgarca', 'Bulgarian')
    df_['language'] = df_['language'].replace('english', eng)
    df_['language'] = df_['language'].replace('Englisch', eng)
    df_['language'] = df_['language'].replace('Türkisch', tr)
    df_['language'] = df_['language'].replace('Türkçe', tr)
    df_['language'] = df_['language'].replace('İNGİLİZCE', eng)
    df_['language'] = df_['language'].replace('italian', ita)
    df_['language'] = df_['language'].replace('ALMANCA', ger)
    df_['language'] = df_['language'].replace('İnglizce', eng)
    df_['language'] = df_['language'].replace('ISPANYOLCA', spa)
    df_['language'] = df_['language'].replace('Español', spa)
    df_['language'] = df_['language'].replace('Azerice', 'Azerbaijani')
    df_['language'] = df_['language'].replace('Germany', ger)
    df_['language'] = df_['language'].replace('Portekizce', 'Portuguese')
    df_['language'] = df_['language'].replace('Çince', chi)
    df_['language'] = df_['language'].replace('Français', fr)
    df_['language'] = df_['language'].replace('Yunanca', 'Greek')
    df_['language'] = df_['language'].replace('Latince', 'Latin')
    df_['language'] = df_['language'].replace('almanca', ger)
    df_['language'] = df_['language'].replace('ispanyolca', spa)
    df_['language'] = df_['language'].replace('INGILIZCE', eng)
    df_['language'] = df_['language'].replace('Italyanca', ita)
    df_['language'] = df_['language'].replace('Deutsche', ger)
    df_['language'] = df_['language'].replace('Turkçe', tr)
    df_['language'] = df_['language'].replace('ENGLISH', eng)
    df_['language'] = df_['language'].replace('Turkce', tr)
    df_['language'] = df_['language'].replace('german', ger)
    df_['language'] = df_['language'].replace('Fransizca', fr)
    df_['language'] = df_['language'].replace('inglizce', eng)
    df_['language'] = df_['language'].replace('Almanca(Beginner)', ger)
    df_['language'] = df_['language'].replace('İngilizice', eng)
    df_['language'] = df_['language'].replace('italyanca', ita)
    df_['language'] = df_['language'].replace('İngilizce,', eng)
    df_['language'] = df_['language'].replace('İngilzce,', eng)
    df_['language'] = df_['language'].replace('Arapça(Temel Düzeyde)', 'Arabic')
    df_['language'] = df_['language'].replace('kürdi', 'Kurdish')
    df_['language'] = df_['language'].replace('İng', eng)
    df_['language'] = df_['language'].replace('İngilizce A2', eng)
    df_['language'] = df_['language'].replace('Engish', eng)
    df_['language'] = df_['language'].replace('İngilice', eng)
    df_['language'] = df_['language'].replace('İngilizce - IELTS 7', eng)
    df_['language'] = df_['language'].replace('İngilizce/English', eng)
    df_['language'] = df_['language'].replace('GERMAN', ger)
    df_['language'] = df_['language'].replace('Turkısh', tr)
    df_['language'] = df_['language'].replace('türkçe', tr)
    df_['language'] = df_['language'].replace('turkish', tr)
    df_['language'] = df_['language'].replace('turkçe', tr)
    df_['language'] = df_['language'].replace('Turksih', tr)
    df_['language'] = df_['language'].replace('rusca', rus)
    df_['language'] = df_['language'].replace('İngilize', eng)
    df_['language'] = df_['language'].replace('İngilzce', eng)
    df_['language'] = df_['language'].replace('İngizce', eng)
    df_['language'] = df_['language'].replace('İngizice', eng)
    df_['language'] = df_['language'].replace('Espanol', spa)
    df_['language'] = df_['language'].replace('İtalian', ita)
    df_['language'] = df_['language'].replace('SPANISH', spa)
    df_['language'] = df_['language'].replace('Kurdî', 'Kurdish')
    df_['language'] = df_['language'].replace('Italien', ita)
    df_['language'] = df_['language'].replace('Türkish', tr)
    df_['language'] = df_['language'].replace('German (Intermediate)', ger)
    df_['language'] = df_['language'].replace('German Language', ger)
    df_['language'] = df_['language'].replace('Almanca B2', ger)
    df_['language'] = df_['language'].replace('İngilizce / English', eng)
    df_['language'] = df_['language'].replace('English, Middle (1100-1500)', eng)
    df_['language'] = df_['language'].replace('İngilizce, Orta (1100-1500)', eng)
    df_['language'] = df_['language'].replace('Almanca, Orta Yüksek (yaklaşık 1050-1500)', ger)
    df_['language'] = df_['language'].replace('German, B2.2 Goethe Instıtut - İZMİR', ger)
    df_['language'] = df_['language'].replace('Fransızca(Université Galatasaray)', 'French')
    df_['language'] = df_['language'].replace('English - Professional working proficiency', eng)
    df_['language'] = df_['language'].replace('Türkçe İşaret Dili', 'Turkish Sign Language')
    df_['language'] = df_['language'].replace('Japonca', jp)
    df_['language'] = df_['language'].replace('Japanese(Beginner)', jp)
    df_['language'] = df_['language'].replace('Japanese (Roomaji)', jp)
    df_['language'] = df_['language'].replace('Japonca | A1', jp)
    df_['language'] = df_['language'].replace('Japanesse(Roomaji)', jp)
    df_['language'] = df_['language'].replace('japonca', jp)
    df_['language'] = df_['language'].replace('germany', ger)
    df_['language'] = df_['language'].replace('English,', eng)
    df_['language'] = df_['language'].replace('German,', ger)
    df_['language'] = df_['language'].replace('Turkish,', tr)
    df_['language'] = df_['language'].replace('3- Turkish', tr)
    df_['language'] = df_['language'].replace('-English', eng)
    df_['language'] = df_['language'].replace('Türkce', tr)
    df_['language'] = df_['language'].replace('2- English', eng)
    df_['language'] = df_['language'].replace('1- French', fr)
    df_['language'] = df_['language'].replace('English US', eng)
    df_['language'] = df_['language'].replace('English UK', eng)
    df_['language'] = df_['language'].replace('2- English', eng)
    df_['language'] = df_['language'].replace('English (US)', eng)
    df_['language'] = df_['language'].replace('English, Advanced', eng)
    df_['language'] = df_['language'].replace('English C1', eng)
    df_['language'] = df_['language'].replace('English, Pre-Advance', eng)
    df_['language'] = df_['language'].replace('English (Upper-Intermediate)', eng)
    df_['language'] = df_['language'].replace('English-B2 Upper Intermediate', eng)
    df_['language'] = df_['language'].replace('■ English ■', eng)
    df_['language'] = df_['language'].replace('English(advanced)', eng)
    df_['language'] = df_['language'].replace('English - (YDS : 93,75)', eng)
    df_['language'] = df_['language'].replace('İngilizce (English)', eng)
    df_['language'] = df_['language'].replace('English (B2, Upper-Intermediate)', eng)
    df_['language'] = df_['language'].replace('English (B2)', eng)
    df_['language'] = df_['language'].replace('Advanced English', eng)
    df_['language'] = df_['language'].replace('English - Global Village Sydney Australia', eng)
    df_['language'] = df_['language'].replace('English (Advanced)', eng)
    df_['language'] = df_['language'].replace('English upper intermediate', eng)
    df_['language'] = df_['language'].replace('İngilizce, İyi', eng)
    df_['language'] = df_['language'].replace('İnglilizce', eng)
    df_['language'] = df_['language'].replace('İngilizce-B1 Wimbledon language academy eğitim sürecindeyim.', eng)
    df_['language'] = df_['language'].replace('İngilizce (C1)', eng)
    df_['language'] = df_['language'].replace('İngilizce (orta)', eng)
    df_['language'] = df_['language'].replace('İngilizce iyi', eng)
    df_['language'] = df_['language'].replace('İngilizce (Advanced)', eng)
    df_['language'] = df_['language'].replace('İngilizce, İyi düzeyde', eng)
    df_['language'] = df_['language'].replace('İngilizce-intermediate', eng)
    df_['language'] = df_['language'].replace('Mesleki İng.', eng)
    df_['language'] = df_['language'].replace('İngilizce, B2', eng)
    df_['language'] = df_['language'].replace('İngilizce(C1)', eng)
    df_['language'] = df_['language'].replace('İngilizce ( TOEIC - 725 )', eng)
    df_['language'] = df_['language'].replace('İngilizce ( orta düzeyde )', eng)
    df_['language'] = df_['language'].replace('İngilizce- BELS english school. as upper intermediate', eng)
    df_['language'] = df_['language'].replace('İngilizce (excellent)', eng)
    df_['language'] = df_['language'].replace('İngilizce, iyi', eng)
    df_['language'] = df_['language'].replace('Mesleki İngilizce', eng)
    df_['language'] = df_['language'].replace('İngiliz', eng)
    df_['language'] = df_['language'].replace('İngilizce Pre-Intermediate', eng)
    df_['language'] = df_['language'].replace('İngilizce(Orta)', eng)
    df_['language'] = df_['language'].replace('İngilizce(Orta Seviye)', eng)
    df_['language'] = df_['language'].replace('İngilizce | B1', eng)
    df_['language'] = df_['language'].replace('İngilizce (Upper-Intermediate)', eng)
    df_['language'] = df_['language'].replace('İngilizce, ileri', eng)
    df_['language'] = df_['language'].replace('İngilizce(B2)', eng)
    df_['language'] = df_['language'].replace('İngilizce (B1 - B2)', eng)
    df_['language'] = df_['language'].replace('İngilizce, Upper Intermediate', eng)
    df_['language'] = df_['language'].replace('İngilizce, B1', eng)
    df_['language'] = df_['language'].replace('İngilizce, İleri (2500-3000)', eng)
    df_['language'] = df_['language'].replace('İngilizce, Orta/İleri Düzey', eng)
    df_['language'] = df_['language'].replace('İngilizce %30', eng)
    df_['language'] = df_['language'].replace('orta düzey İngilizce', eng)
    df_['language'] = df_['language'].replace('Französisch', fr)
    df_['language'] = df_['language'].replace('France', fr)
    df_['language'] = df_['language'].replace('Francais', fr)
    df_['language'] = df_['language'].replace('Fransa', fr)
    df_['language'] = df_['language'].replace('Fransız', fr)
    df_['language'] = df_['language'].replace('Italiano', ita)
    df_['language'] = df_['language'].replace('Chinese (Simplified)', chi)
    df_['language'] = df_['language'].replace('Elementary Chinese', chi)
    df_['language'] = df_['language'].replace('Chinese(Simplified-Mandarin) - 中文', chi)
    df_['language'] = df_['language'].replace('Çince (Mandarin)', chi)
    df_['language'] = df_['language'].replace('Çince (Basitleştirilmiş)', chi)
    df_['language'] = df_['language'].replace('Çinçe', chi)
    df_['language'] = df_['language'].replace('Türkçe,', tr)
    df_['language'] = df_['language'].replace('Engilish,', eng)
    df_['language'] = df_['language'].replace('Makedonca', mak)
    df_['language'] = df_['language'].replace('Makedonski', mak)
    df_['language'] = df_['language'].replace('Macarca', 'Hungarian')
    df_['language'] = df_['language'].replace('Litvanyaca', 'Lithuanian')
    df_['language'] = df_['language'].replace('Korece | A2', 'Korean')
    df_['language'] = df_['language'].replace('Deutsch', ger)

    df_.loc[df_['language'] == 'ingilizce(english)', 'language'] = eng
    df_.loc[df_['language'] == 'Almanca(A1)', 'language'] = ger
    df_.loc[df_['language'] == 'Almanca (Düşük Seviye)', 'language'] = ger
    df_.loc[df_['language'] == 'Almanca (başlangıç)', 'language'] = ger
    df_.loc[df_['language'] == 'Almanca/German', 'language'] = ger
    df_.loc[df_['language'] == 'Almanca / Deutsch', 'language'] = ger
    df_.loc[df_['language'] == 'Almanca (IAnfänger A2) (Elementary)', 'language'] = ger
    df_.loc[df_['language'] == 'Almanca (basic)', 'language'] = ger
    df_.loc[df_['language'] == 'German (Deutsches Sprachdiplom - 2.Stufe)', 'language'] = ger
    df_.loc[df_['language'] == 'Germanic languages', 'language'] = ger
    df_.loc[df_['language'] == 'German(beginner)', 'language'] = ger
    df_.loc[df_['language'] == 'German (A2)', 'language'] = ger
    df_.loc[df_['language'] == 'German (Beginner)', 'language'] = ger
    df_.loc[df_['language'] == 'Gerrman', 'language'] = ger
    df_.loc[df_['language'] == 'CHINESE', 'language'] = chi
    df_.loc[df_['language'] == 'turkce', 'language'] = tr
    df_.loc[df_['language'] == 'Dutch (beginner)', 'language'] = 'Dutch'
    df_.loc[df_['language'] == 'arapça', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'arapca', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'arabic', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Arabic', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Arapca', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Arabic (only very basic speaking skills)', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Arabe', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Arapça-A1-A2', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Arabish', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'South Sudanese Arabic', 'language'] = 'Arabic'
    df_.loc[df_['language'] == 'Başlangıç seviyesinde İspanyolca', 'language'] = spa
    df_.loc[df_['language'] == 'İspanyolca, Başlangıç', 'language'] = spa
    df_.loc[df_['language'] == 'italiano', 'language'] = ita
    
    df_ = df_.drop_duplicates()
    most_freq_langs = df_['language'].value_counts().keys()[:size].tolist()
    grouped = df_.groupby(by='user_id', as_index=False).agg({'language': 'unique'})
    for lang in tqdm(most_freq_langs):
        grouped[f'{seperator}_{lang}'] = grouped['language'].apply(lambda x: 1 if lang in x else 0)

    grouped = grouped.merge(df_.groupby(by='user_id', as_index=False).agg(
        total_languages=('language', 'count')), on=['user_id'], how='left')
    grouped = grouped.drop(columns=['language'], axis=1)

    #prof_df = df_.groupby(by = ['user_id', 'proficiency'], as_index = False).agg({'language': 'count'})
    #pv = pd.pivot_table(prof_df, values='language', index=['user_id'], columns=['proficiency'], aggfunc=np.sum, fill_value=0).reset_index()
    #pv.columns.name = None
    #pv.columns = pv.columns.map(lambda x: f'proficiency_{str(x)}' if x != 'user_id' else x)

    #grouped = grouped.merge(pv, on = ['user_id'], how = 'left')

    return grouped

In [244]:
#df = load_languages('../../../datasets/garanti-bbva-data-camp/languages.csv')

#df

100%|██████████| 12/12 [00:01<00:00,  7.53it/s]


,user_id,language_English,language_Turkish,language_German,language_Spanish,language_French,language_Russian,language_Arabic,language_Italian,language_Japanese,language_Azerbaijani,language_Chinese,language_Korean,total_languages,proficiency_elementary,proficiency_full_professional,proficiency_limited_working,proficiency_native_or_bilingual,proficiency_professional_working
0,8,1,1,0,0,1,0,0,0,0,0,0,0,3,1.0,1.0,0.0,1.0,0.0
1,10,1,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
2,11,1,1,0,0,0,0,0,0,0,0,0,0,3,0.0,0.0,0.0,2.0,1.0
3,12,1,1,0,0,0,0,0,0,0,0,0,0,2,0.0,0.0,0.0,1.0,1.0
4,13,1,0,1,0,0,0,0,0,0,0,0,0,2,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37285,66265,1,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
37286,66269,1,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
37287,66271,1,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN
37288,66272,1,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,1.0
